In [1]:
from PIL import Image, ImageDraw, ImageOps # Adicionado ImageOps aqui
import random
import os
import math
import json
import numpy as np 
import cv2         

# --- CONFIGURAÇÕES PRINCIPAIS ---
imagem_limpa_base = "Base/Placa.jpg"
imagens_referencia_cor = ["Base/Cor1.jpg", "Base/Cor2.jpg", "Base/Cor3.jpg"]
pastas_saida = ["sequencia_sujeira_tipo1", "sequencia_sujeira_tipo2", "sequencia_sujeira_tipo3", "sequencia_sujeira_misturada"]
prefixos_arquivo = ["placa_tipo1", "placa_tipo2", "placa_tipo3", "placa_misturada"]

# --- CONTROLES GERAIS DE PROGRESSÃO E REALISMO ---
numero_de_etapas = 30
porcentagem_sujeira_por_etapa = 2.5 
tamanho_min_mancha, tamanho_max_mancha = 1, 2
opacidade_min_mancha, opacidade_max_mancha = 30, 220
chance_cluster, area_cluster = 0.2, 1

# --------------------------------------------------------------------

# --- ETAPA 1: CARREGAR ÁREA, CALCULAR PONTOS E CARREGAR PALETAS ---
try:
    with open("area_selecionada.json", "r") as f:
        data = json.load(f)
    contour = np.array(data['points'], dtype=np.int32)
    x, y, w, h = cv2.boundingRect(contour)
    bounding_box = {"x1": x, "y1": y, "x2": x + w, "y2": y + h}
    
    area_do_poligono = cv2.contourArea(contour)
    if area_do_poligono <= 0:
        print("ERRO CRÍTICO: A área do polígono selecionado é zero ou inválida.")
        print("Por favor, execute 'definir_area_poligono.py' e desenhe uma área válida.")
        exit()

    pontos_por_etapa = int(area_do_poligono * (porcentagem_sujeira_por_etapa / 100.0))
    pontos_por_etapa = max(1, pontos_por_etapa) 
    
    print("Arquivo de área 'area_selecionada.json' carregado.")
    print(f"Área do polígono calculada: {area_do_poligono:.2f} pixels.")
    print(f"Com a configuração de {porcentagem_sujeira_por_etapa}%, serão adicionados {pontos_por_etapa} pontos de sujeira a cada etapa.")

except FileNotFoundError:
    print("ERRO CRÍTICO: O arquivo 'area_selecionada.json' não foi encontrado.")
    print("Por favor, execute o script 'definir_area_poligono.py' primeiro para selecionar a área.")
    exit()

# ... (Lógica para carregar paletas de cores) ...
lista_de_paletas = []
for path in imagens_referencia_cor:
    try:
        img = Image.open(path).convert("RGB")
        img.thumbnail((150, 150))
        lista_de_paletas.append(list(img.getdata()))
    except FileNotFoundError:
        print(f"ERRO: A imagem de textura '{path}' não foi encontrada.")
        exit()
paleta_misturada = sum(lista_de_paletas, [])
lista_de_paletas.append(paleta_misturada)

def get_random_point_in_polygon(contour, bbox):
    while True:
        rand_x = random.randint(bbox["x1"], bbox["x2"])
        rand_y = random.randint(bbox["y1"], bbox["y2"])
        if cv2.pointPolygonTest(contour, (rand_x, rand_y), False) >= 0:
            return rand_x, rand_y

# --- ETAPA 2: LOOP PRINCIPAL DE GERAÇÃO (MODIFICADO) ---
# Carrega a imagem limpa
pil_img_base = Image.open(imagem_limpa_base)
# Aplica a correção de orientação EXIF para evitar que a imagem fique "de lado"
img_base = ImageOps.exif_transpose(pil_img_base).convert("RGB")
# --- FIM DA ALTERAÇÃO ---

largura, altura = img_base.size

for i in range(len(pastas_saida)):
    pasta_atual, prefixo_atual, paleta_atual = pastas_saida[i], prefixos_arquivo[i], lista_de_paletas[i]
    print(f"\nINICIANDO GERAÇÃO PARA: '{pasta_atual}'")
    os.makedirs(pasta_atual, exist_ok=True)
    
    img_progressiva = img_base.copy()
    draw = ImageDraw.Draw(img_progressiva, 'RGBA')
    ultimas_posicoes = []

    for etapa in range(1, numero_de_etapas + 1):
        print(f" -> Gerando Etapa {etapa}/{numero_de_etapas}...", end='\r')
        
        manchas_nesta_etapa = pontos_por_etapa

        for _ in range(manchas_nesta_etapa):
            if ultimas_posicoes and random.random() < chance_cluster:
                last_x, last_y = random.choice(ultimas_posicoes)
                angle, radius = random.uniform(0, 2*math.pi), random.uniform(0, area_cluster)
                x = int(last_x + radius * math.cos(angle))
                y = int(last_y + radius * math.sin(angle))
                
                if cv2.pointPolygonTest(contour, (x, y), False) < 0:
                    x, y = get_random_point_in_polygon(contour, bounding_box)
            else:
                x, y = get_random_point_in_polygon(contour, bounding_box)
            
            ultimas_posicoes.append((x, y))
            if len(ultimas_posicoes) > 100: ultimas_posicoes.pop(0)

            tamanho = random.randint(tamanho_min_mancha, tamanho_max_mancha)
            opacidade = random.randint(opacidade_min_mancha, opacidade_max_mancha)
            cor_rgb = random.choice(paleta_atual)
            cor_rgba = cor_rgb + (opacidade,)
            
            x_f, y_f = x - tamanho//2, y - tamanho//2
            draw.rectangle([x_f, y_f, x_f + tamanho - 1, y_f + tamanho - 1], fill=cor_rgba)

        nome_arquivo = f"{prefixo_atual}_etapa_{etapa:02d}.png"
        caminho_saida = os.path.join(pasta_atual, nome_arquivo)
        img_progressiva.save(caminho_saida)
    
    print(f"\n -> Geração para '{pasta_atual}' concluída.                        ")

print("\n\nPROCESSO FINALIZADO!")

Arquivo de área 'area_selecionada.json' carregado.
Área do polígono calculada: 563690.00 pixels.
Com a configuração de 2.5%, serão adicionados 14092 pontos de sujeira a cada etapa.

INICIANDO GERAÇÃO PARA: 'sequencia_sujeira_tipo1'
 -> Gerando Etapa 30/30...
 -> Geração para 'sequencia_sujeira_tipo1' concluída.                        

INICIANDO GERAÇÃO PARA: 'sequencia_sujeira_tipo2'
 -> Gerando Etapa 30/30...
 -> Geração para 'sequencia_sujeira_tipo2' concluída.                        

INICIANDO GERAÇÃO PARA: 'sequencia_sujeira_tipo3'
 -> Gerando Etapa 30/30...
 -> Geração para 'sequencia_sujeira_tipo3' concluída.                        

INICIANDO GERAÇÃO PARA: 'sequencia_sujeira_misturada'
 -> Gerando Etapa 30/30...
 -> Geração para 'sequencia_sujeira_misturada' concluída.                        


PROCESSO FINALIZADO!
